In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from astropy import units
from astropy.constants import G
import importlib

import dataset_compute
import snapshot_obj

In [ ]:
importlib.reload(dataset_compute)
importlib.reload(snapshot_obj)

## Construct data dictionary

Add entries for each simulation, and specify M31 and MW galaxies:

In [ ]:
snap_id = 127
sim_ids = ["V1_LR_fix", "V1_LR_curvaton_p082_fix", "V1_LR_curvaton_p084_fix"]
names = ["LCDM", "p082", "p084"]
direc = "LowResolution"

# Define M31 and MW in each simulation:
m31 = [(1,0), (1,0), (1,0)]
mw = [(2,0), (1,1), (1,0)]

data = {}
for name, sim_id, m31_ns, mw_ns in zip(names, sim_ids, m31, mw):
    data[name] = {"snapshot": snapshot_obj.Snapshot(sim_id, snap_id, name=name),
                  "M31_identifier": m31_ns,
                  "MW_identifier": mw_ns}

In [ ]:
snap_id = 127
sim_ids = ["V1_MR_fix", "V1_MR_curvaton_p082_fix"]
names = ["plain-LCDM", "curv-p082"]
direc = "MediumResolution"

# Define M31 and MW in each simulation:
m31 = [(1,0), (1,0)]
mw = [(2,0), (1,1)]

data = {}
for name, sim_id, m31_ns, mw_ns in zip(names, sim_ids, m31, mw):
    data[name] = {"snapshot": snapshot_obj.Snapshot(sim_id, snap_id, name=name),
                  "M31_identifier": m31_ns,
                  "MW_identifier": mw_ns}

Choose how to distinguish between satellite and isolated galaxies:

In [ ]:
distinction = "by_gn"

Read datasets, split into satellites and isolated galaxies, and add to the data dictionary. We also disregard dark halos and potential spurious halos with $v_\mathrm{max} = 0$.

In [ ]:
for name, sim_data in data.items():
    # Get data:
    snap = sim_data["snapshot"]
    cops = snap.get_subhalos("CentreOfPotential") * units.cm.to(units.Mpc)
    mass = snap.get_subhalos("Vmax") * units.g.to(units.Msun)
    
    m31 = sim_data["M31_identifier"]
    mw = sim_data["MW_identifier"]
    
    # Split into satellites:
    if distinction == "by_r":
        masks_sat, mask_isol = dataset_compute.split_satellites_by_distance(
            snap, m31, mw)
    elif distinction == "by_gn":
        masks_sat, mask_isol = dataset_compute.split_satellites_by_group_number(
            snap, m31, mw)
    
    mask_nonzero_vmax = dataset_compute.prune_vmax(snap)

    # Add to dictionary:
    data[name]["LG_centre"] = dataset_compute.compute_LG_centre(snap, m31, mw) \
        * units.cm.to(units.Mpc)
    m31_idx = snap.index_of_halo(m31[0], m31[1])
    mw_idx = snap.index_of_halo(mw[0], mw[1])
    for dataset_name, dataset in zip(["COP", "Mass"], [cops, mass]):
        data[name][dataset_name] = \
        {"M31": dataset[m31_idx],
         "MW": dataset[mw_idx],
         "M31_satellites": dataset[np.logical_and(masks_sat[0], mask_nonzero_vmax)],
         "MW_satellites": dataset[np.logical_and(masks_sat[1], mask_nonzero_vmax)],
         "isolated": dataset[np.logical_and(mask_isol, mask_nonzero_vmax)]
        }

In [ ]:
# Construct saving location:
filename = 'satellite_locations_with_mass_markers'
for name in names:
    filename += "_{}".format(name)
filename += ".png"
    
home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures", direc)
filename = os.path.join(path, filename)

In [ ]:
projs = [["x", "y", 0, 1],
         ["x", "z", 0, 2]]

fig, axes = plt.subplots(sharey='row', sharex='col', figsize=(10,7),
                         ncols=len(data), nrows=len(projs))
plt.subplots_adjust(wspace=0.04)
plt.subplots_adjust(hspace=0.04)

axes[1,0].set_xlabel("$x [\mathrm{Mpc}]$")
axes[1,1].set_xlabel("$x [\mathrm{Mpc}]$")
axes[0,0].set_ylabel("$y [\mathrm{Mpc}]$")
axes[1,0].set_ylabel("$z [\mathrm{Mpc}]$")

centre = np.mean(np.vstack(
    [sim_data["LG_centre"] for sim_data in data.values()]),
                 axis=0)

# Iterate through simulations (columns):
for i, (key, sim_data) in enumerate(data.items()):
    m31 = sim_data["M31_identifier"]
    mw = sim_data["MW_identifier"]
    m31_idx = snap.index_of_halo(m31[0], m31[1])
    mw_idx = snap.index_of_halo(mw[0], mw[1])
    
    axes[0, i].set_title(key)
    
    smin = 0.01; smax = 3
    all_dwarfs = np.concatenate([sim_data["Mass"]["isolated"], 
                                sim_data["Mass"]["M31_satellites"],
                                sim_data["Mass"]["MW_satellites"]])
    mmax = np.min(all_dwarfs)
    mmin = np.max(all_dwarfs)
    
    def transf(x):
        return 1/(mmax-mmin) * (smin*(x-mmin) - smax*(x-mmax))
    
    # Iterate through projections (rows):   
    ax_size = 5
    for j, ((x, x_idx), (y, y_idx)) in enumerate(
        zip([("x", 0), ("x", 0)], [("y", 1), ("z", 2)])):
        
        axes[j, i].set_xlim(centre[x_idx] - ax_size/2, 
                            centre[x_idx] + ax_size/2)
        axes[j, i].set_ylim(centre[y_idx] - ax_size/2, 
                            centre[y_idx] + ax_size/2)
        
        axes[j, i].scatter(sim_data["COP"]['isolated'][:,x_idx], 
                           sim_data["COP"]['isolated'][:,y_idx], 
                           c='gray', 
                           s=transf(sim_data["Mass"]["isolated"]))
        axes[j, i].scatter(sim_data["COP"]['M31_satellites'][:,x_idx], 
                           sim_data["COP"]['M31_satellites'][:,y_idx], 
                           c='pink', 
                           s=transf(sim_data["Mass"]["M31_satellites"]))
        axes[j, i].scatter(sim_data["COP"]['MW_satellites'][:,x_idx], 
                           sim_data["COP"]['MW_satellites'][:,y_idx], 
                           c='lightblue', 
                           s=transf(sim_data["Mass"]["MW_satellites"]))
        
        m31_cop = sim_data["COP"]['M31']
        axes[j, i].scatter(m31_cop[x_idx], m31_cop[y_idx], c='red', s=smax)
        mw_cop = sim_data["COP"]['MW']
        axes[j, i].scatter(mw_cop[x_idx], mw_cop[y_idx], c='blue', s=smax)
        
plt.savefig(filename, dpi=300)